In [1]:
import os
import subprocess
import cv2

In [2]:
isymbols = 'symbols.txt'
osymbols = 'symbols.txt'

In [3]:
def create_fsa(word):
    file_name = word + '_fsa.txt'
    file_fsa = open(file_name, 'w')
    for i in range(len(word)):
        file_fsa.write('{} {} {}\n'.format(i, i + 1, word[i]))
    len_of_word = str(len(word))
    file_fsa.write(len_of_word)
    file_fsa.close

    process = subprocess.Popen(['fst\\fstcompile', '-acceptor', '-isymbols={}'.format(isymbols), '{}_fsa.txt'.format(word), '{}.fsa'.format(word)], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    print('finished create fsa for', word)

In [4]:
create_fsa('cat')

finished create fsa for cat


In [5]:
def create_fst(string):
	s, morp = string.split(' -> ')

	f = open('{}_fst.txt'.format(s), 'w')
	word = s
	parts = morp.split('+')
	d = len(s) - (len(parts[0]) + len(parts) - 1)
	if d>0:
		parts[0] += '#' * d
	else:
		d = -d
		s += '#' * d

	for i in range(len(parts[0])):
		f.write('{} {} {} {}\n'.format(i, i+1, s[i], morp[i]))

	morplen = len(parts[0])
	for  i in range(1,len(parts)):
		parts[i] = '+' + parts[i]
		f.write('{} {} {} {}\n'.format(morplen + i - 1, morplen + i, s[morplen + i - 1], parts[i]))

	f.write('{}\n'.format(morplen + len(parts) - 1))

	f.close()

	process = subprocess.Popen(['fst\\fstcompile', '-isymbols={}'.format(isymbols), '-osymbols={}'.format(osymbols), '{}_fst.txt'.format(word), '{}.fst'.format(word)], stdout = subprocess.PIPE, stderr = subprocess.PIPE)

	print('create fst for', string)


In [6]:
create_fst('cat -> cat+N+SG')

create fst for cat -> cat+N+SG


In [7]:
def union(word1, word2, type, output_file_name):
    process = subprocess.Popen(['fst\\fstunion', f'{word1}.{type}', f'{word2}.{type}', f'{word1}{word2}.{type}'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    process = subprocess.Popen(['fst\\fstdeterminize', f'{word1}{word2}.{type}', f'{output_file_name}.{type}'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    process = subprocess.Popen(['fst\\fstrmepsilon', f'{output_file_name}.{type}', f'temp.{type}'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    process = subprocess.Popen(['fst\\fstdeterminize', f'temp.{type}', f'{output_file_name}.{type}'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    process = subprocess.Popen(['fst\\fstminimize', f'{output_file_name}.{type}'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)

In [8]:
def show(file_name, type, type_of_image = 'jpg'):
    if type == 'fsa':
        process = subprocess.Popen(['fst\\fstdraw', '-acceptor', '-portrait', '-isymbols=symbols.txt', '-osymbols=symbols.txt', f'{file_name}.fsa', f'{file_name}.dot'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    else:
        process = subprocess.Popen(['fst\\fstdraw', '-portrait', '-isymbols=symbols.txt', '-osymbols=symbols.txt', f'{file_name}.fst', f'{file_name}.dot'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    
    process = subprocess.Popen(['graphviz\\dot', f'-T{type_of_image}', f'{file_name}.dot', f'-o{file_name}.{type_of_image}'], stdout = subprocess.PIPE, stderr = subprocess.PIPE)

    os.system(f'{file_name}.{type_of_image}')


In [9]:
create_fst('cat -> cat+N+SG')
create_fst('caps -> cap+N+PL')
create_fst('captain -> captain+N+SG')

union('cat','caps','fst','catcaps')
union('catcaps','captain','fst','catcapscaptain')
show('catcapscaptain','fst')

create fst for cat -> cat+N+SG
create fst for caps -> cap+N+PL
create fst for captain -> captain+N+SG


In [ ]:
cv2.imshow()